# Exoplanetas

### Pasando de pandas a SQL

In [ ]:
def crear_db(nombre, user='root', password='password', check=True):
    
    crea_db=conn.connect(host='localhost', user=user, passwd=password)

    cursor=crea_db.cursor()

    cursor.execute(f'drop database if exists {nombre}')
    cursor.execute(f'create database {nombre}')
    
    print('Hecho, base de datos creada. ;)')
    
    if check:
        cursor.execute('show databases')
        for x in cursor:
            print(x)

In [ ]:
def crear_tabla_desde_df(df, nombre_tabla, nombre_db, user='root', password='password', check=True, drop=False):
    
    db=conn.connect(host='localhost', user=user, passwd=password, database=nombre_db)
    cursor=db.cursor()
    
    if drop:
        cursor.execute(f'drop table if exists {nombre_tabla}')
    
    tabla=f"create table {nombre_tabla}(id int, {' varchar(200),'.join(df.columns)} varchar(200));"
    print(tabla)
    
    cursor.execute(tabla)
    
    print(f'Hecho, tabla {nombre_tabla} creada. ;)')
    
    if check:
        cursor.execute('show tables')
        for x in cursor:
            print(x)

In [ ]:
def insertar_datos(df, nombre_tabla, nombre_db, user='root', password='password', check=True):
    
    db=conn.connect(host='localhost', user=user, passwd=password, database=nombre_db)
    cursor=db.cursor()
    
    for i in range(len(df)):
    
        insert_query='insert into {} (id, {}) values {};'\
                     .format(nombre_tabla, ','.join(df.columns), tuple(np.insert(df.iloc[i].values, 0, i)))

        cursor.execute(insert_query)

    db.commit()
    
    print(f'Hecho, datos dentro de tabla {nombre_tabla}. ;)')
    print()
    print(insert_query)

In [ ]:
def leer_sql(query, nombre_tabla, nombre_db, user='root', password='password', check=True):
    
    db=conn.connect(host='localhost', user=user, passwd=password, database=nombre_db)
    
    data=pd.read_sql(query, db)
    
    return data

### 1ª Fuente - Kaggle (formato csv)

https://www.kaggle.com/nasa/kepler-exoplanet-search-results

**A sql...**

### 2ª Fuente - Exoplanet API (formato libreria)


http://exoplanet.eu/API/

### 3ª Fuente - Wikipedia (formato scrapeo)

https://en.wikipedia.org/wiki/List_of_exoplanets_discovered_using_the_Kepler_space_telescope

### Fusión de los datos

**similitud de strings**

https://github.com/seatgeek/fuzzywuzzy